# Algorithms Lab: Searching & Sorting (College-Level)

Welcome! This notebook is a guided, hands-on lab to help you **understand and implement fundamental algorithms**.

You'll learn by doing:
- Big-O reasoning via small experiments
- Linear search vs. binary search
- Classic sorts (selection, insertion, merge, quick)
- Stability & in-place properties
- Comparator-based sorting & key functions
- Practice problems (LeetCode-style) with tests

**Workflow suggestion** (for GitHub classroom / repo):
1. Create a feature branch named `algorithms-lab`.
2. Work through each section, filling in `TODO` blocks.
3. Run the local tests (cells named **Run tests**). All tests should pass.
4. Commit your changes and open a Pull Request.

> Pro tip: Write small helper functions, keep code clean, add docstrings, and handle edge cases.


## 0) Setup & Utilities
Run this once. It defines helpers and a small testing harness.

In [2]:
from __future__ import annotations
import math, random, time
from dataclasses import dataclass
from typing import List, Callable, Any, Optional, Iterable, Tuple

def assert_equal(actual, expected, msg: str = ""):
    if actual != expected:
        raise AssertionError(f"Expected {expected!r} but got {actual!r}. {msg}")

def assert_true(expr: bool, msg: str = ""):
    if not expr:
        raise AssertionError(f"Assertion failed. {msg}")

def time_function(fn: Callable, *args, repeats: int = 5, **kwargs) -> float:
    """Return median runtime (seconds) over several repeats."""
    times = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        fn(*args, **kwargs)
        times.append(time.perf_counter() - t0)
    times.sort()
    return times[len(times)//2]


## 1) Warm-up: Big-O by Experiment

You're given two functions that do the same job but with different complexities.
Implement them, then **measure** how their runtime scales with input size.

**Task:**
- Implement `sum_pairs_quadratic` in $O(n^2)$.
- Implement `sum_pairs_linear` in $O(n)$ by aggregating counts.
- Verify scaling with the `benchmark_sum_pairs` cell.


In [3]:
def sum_pairs_quadratic(nums: List[int]) -> int:
    """Return the number of (i, j) pairs with i < j and nums[i] == nums[j].
    Expected complexity: O(n^2).
    """
    # TODO: brute-force nested loops
    count = 0
    # --- your code here ---
    for i in range(len(nums)):
        for j in range(i+1, len(nums)):
            if nums[i] == nums[j]:
                count += 1
    return count

def sum_pairs_linear(nums: List[int]) -> int:
    """Return the number of equal pairs in linear time using a hashmap.
    Expected complexity: O(n).
    """
    # TODO: linear-time using counts
    counts = {}
    for x in nums:
        counts[x] = counts.get(x, 0) + 1
    total = 0
    for c in counts.values():
        # number of pairs from c identical items = c choose 2
        total += c*(c-1)//2
    return total


In [4]:
# Run tests (Warm-up)
assert_equal(sum_pairs_quadratic([1,1,2]), 1)
assert_equal(sum_pairs_quadratic([1,1,1]), 3)  # pairs: (0,1),(0,2),(1,2)
assert_equal(sum_pairs_linear([1,1,2]), 1)
assert_equal(sum_pairs_linear([1,1,1]), 3)
print("[ok] Warm-up tests passed.")


[ok] Warm-up tests passed.


In [5]:
# Benchmark (feel free to tweak sizes)
for n in [1_000, 2_000, 4_000, 8_000]:
    data = [random.randint(0, 1000) for _ in range(n)]
    t_quad = time_function(sum_pairs_quadratic, data, repeats=1)  # may get slow
    t_lin = time_function(sum_pairs_linear, data)
    print(f"n={n:6d} | quadratic ~ {t_quad:7.4f}s | linear ~ {t_lin:7.4f}s")


n=  1000 | quadratic ~  0.0220s | linear ~  0.0001s
n=  2000 | quadratic ~  0.0760s | linear ~  0.0002s
n=  4000 | quadratic ~  0.3132s | linear ~  0.0003s
n=  8000 | quadratic ~  1.2753s | linear ~  0.0005s


## 2) Searching

### 2.1 Linear Search
Implement a simple linear scan. Return the index or `-1` if not found.

### 2.2 Binary Search
Assume the input list is **sorted ascending**. Implement both **iterative** and **recursive** versions. Return the index or `-1`.


In [6]:
def linear_search(arr: List[int], target: int) -> int:
    # TODO: implement linear scan
    for i, x in enumerate(arr):
        if x == target:
            return i
    return -1

def binary_search_iter(arr: List[int], target: int) -> int:
    # TODO: iterative binary search
    lo, hi = 0, len(arr) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            lo = mid + 1
        else:
            hi = mid - 1
    return -1

def binary_search_rec(arr: List[int], target: int, lo: int = 0, hi: Optional[int] = None) -> int:
    # TODO: recursive binary search
    if hi is None:
        hi = len(arr) - 1
    if lo > hi:
        return -1
    mid = (lo + hi) // 2
    if arr[mid] == target:
        return mid
    elif arr[mid] < target:
        return binary_search_rec(arr, target, mid+1, hi)
    else:
        return binary_search_rec(arr, target, lo, mid-1)


In [7]:
# Run tests (Searching)
arr = [1,3,5,7,9,11]
assert_equal(linear_search(arr, 7), 3)
assert_equal(linear_search(arr, 2), -1)
assert_equal(binary_search_iter(arr, 1), 0)
assert_equal(binary_search_iter(arr, 11), 5)
assert_equal(binary_search_iter(arr, 4), -1)
assert_equal(binary_search_rec(arr, 1), 0)
assert_equal(binary_search_rec(arr, 11), 5)
assert_equal(binary_search_rec(arr, 4), -1)
print("[ok] Search tests passed.")


[ok] Search tests passed.


## 3) Sorting

You'll implement four classic algorithms. For each, preserve the function signature. Don't use Python's built-ins for the core logic.

**Tasks**
- `selection_sort` (in-place, not stable)
- `insertion_sort` (in-place, stable)
- `merge_sort` (out-of-place, stable)
- `quick_sort` (in-place partition, average O(n log n))

> **Stability reminder:** Stable sorts keep the original relative order of items that compare equal.


In [7]:
def selection_sort(arr: List[int]) -> List[int]:
    a = arr[:]  # copy; okay to do in-place as well
    n = len(a)
    for i in range(n):
        min_idx = i
        for j in range(i+1, n):
            if a[j] < a[min_idx]:
                min_idx = j
        a[i], a[min_idx] = a[min_idx], a[i]
    return a

def insertion_sort(arr: List[int]) -> List[int]:
    a = arr[:]  # working copy
    for i in range(1, len(a)):
        key = a[i]
        j = i - 1
        while j >= 0 and a[j] > key:
            a[j+1] = a[j]
            j -= 1
        a[j+1] = key
    return a

def merge_sort(arr: List[int]) -> List[int]:
    # Top-down recursive merge sort
    if len(arr) <= 1:
        return arr[:]
    mid = len(arr)//2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    # merge phase (stable)
    i=j=0
    out = []
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:  # <= makes it stable
            out.append(left[i]); i+=1
        else:
            out.append(right[j]); j+=1
    out.extend(left[i:])
    out.extend(right[j:])
    return out

def quick_sort(arr: List[int]) -> List[int]:
    a = arr[:]
    def partition(lo, hi):
        pivot = a[hi]
        i = lo
        for j in range(lo, hi):
            if a[j] <= pivot:
                a[i], a[j] = a[j], a[i]
                i += 1
        a[i], a[hi] = a[hi], a[i]
        return i
    def _qs(lo, hi):
        if lo < hi:
            p = partition(lo, hi)
            _qs(lo, p-1)
            _qs(p+1, hi)
    _qs(0, len(a)-1)
    return a


In [8]:
# Run tests (Sorting)
samples = [
    [], [1], [2,1], [3,2,1], [5,1,4,2,8],
]
for s in samples:
    exp = sorted(s)
    assert_equal(selection_sort(s), exp, "selection_sort failed")
    assert_equal(insertion_sort(s), exp, "insertion_sort failed")
    assert_equal(merge_sort(s), exp, "merge_sort failed")
    assert_equal(quick_sort(s), exp, "quick_sort failed")
print("[ok] Sorting tests passed.")


[ok] Sorting tests passed.


### 3.1 Sorting with Custom Keys (Comparators)
Python's `sorted` accepts a `key` function. Implement a small stable sort that accepts a `key`.

**Task:** Implement `stable_sort_by_key` using **insertion sort** (to guarantee stability) and the provided `key`.


In [9]:
def stable_sort_by_key(arr: List[Any], key: Callable[[Any], Any]) -> List[Any]:
    a = arr[:]  # copy
    # TODO: insertion sort using key(x) comparisons to keep it stable
    for i in range(1, len(a)):
        item = a[i]
        k = key(item)
        j = i - 1
        while j >= 0 and key(a[j]) > k:
            a[j+1] = a[j]
            j -= 1
        a[j+1] = item
    return a


In [10]:
# Run tests (Custom key & stability)
people = [("alice", 3), ("bob", 3), ("carl", 2), ("dana", 3)]
res = stable_sort_by_key(people, key=lambda x: x[1])
assert_equal(res, [("carl", 2), ("alice", 3), ("bob", 3), ("dana", 3)])
# stability check: among ties (3), original order alice -> bob -> dana preserved
print("[ok] Stability test passed.")


[ok] Stability test passed.


## 4) Practice Problems (LeetCode-style)
Solve each with the **best possible time complexity**.

1. **Two Sum (classic)** – Return indices of two numbers adding up to target. (Hash map)
2. **Find First Bad Version** – Given `is_bad(version)` (monotonic), find first bad (Binary search).
3. **Search Rotated Sorted Array** – Return index of target in rotated sorted array (Binary search variant).
4. **Kth Largest Element** – Return the k-th largest using Quickselect (average O(n)).
5. **Merge Intervals** – Given intervals, merge overlaps (Sort + linear sweep).


In [11]:
# 4.1 Two Sum
def two_sum(nums: List[int], target: int) -> List[int]:
    # TODO: return [i, j] with i<j
    seen = {}
    for i, x in enumerate(nums):
        need = target - x
        if need in seen:
            return [seen[need], i]
        seen[x] = i
    return []

# 4.2 First Bad Version (binary search). We'll simulate is_bad.
def first_bad_version(n: int, is_bad: Callable[[int], bool]) -> int:
    # TODO: return smallest v in [1..n] with is_bad(v)=True (assume exists)
    lo, hi = 1, n
    while lo < hi:
        mid = (lo + hi) // 2
        if is_bad(mid):
            hi = mid
        else:
            lo = mid + 1
    return lo

# 4.3 Search Rotated Sorted Array
def search_rotated(nums: List[int], target: int) -> int:
    # TODO: return index or -1
    lo, hi = 0, len(nums)-1
    while lo <= hi:
        mid = (lo + hi) // 2
        if nums[mid] == target:
            return mid
        # left half sorted
        if nums[lo] <= nums[mid]:
            if nums[lo] <= target < nums[mid]:
                hi = mid - 1
            else:
                lo = mid + 1
        else:  # right half sorted
            if nums[mid] < target <= nums[hi]:
                lo = mid + 1
            else:
                hi = mid - 1
    return -1

# 4.4 Kth Largest via Quickselect
def kth_largest(nums: List[int], k: int) -> int:
    # TODO: quickselect in O(n) expected
    arr = nums[:]
    k_small = len(arr) - k  # kth largest == index for kth smallest
    lo, hi = 0, len(arr)-1
    def partition(lo, hi):
        pivot = arr[hi]
        i = lo
        for j in range(lo, hi):
            if arr[j] <= pivot:
                arr[i], arr[j] = arr[j], arr[i]
                i += 1
        arr[i], arr[hi] = arr[hi], arr[i]
        return i
    while True:
        p = partition(lo, hi)
        if p == k_small:
            return arr[p]
        elif p < k_small:
            lo = p + 1
        else:
            hi = p - 1

# 4.5 Merge Intervals
def merge_intervals(intervals: List[List[int]]) -> List[List[int]]:
    # TODO: merge overlapping intervals
    if not intervals:
        return []
    intervals = sorted(intervals)
    out = [intervals[0]]
    for s, e in intervals[1:]:
        last = out[-1]
        if s <= last[1]:
            last[1] = max(last[1], e)
        else:
            out.append([s, e])
    return out


In [12]:
# Run tests (Practice)
assert_equal(two_sum([2,7,11,15], 9), [0,1])

bad_after = 7
def is_bad(v): return v >= bad_after
assert_equal(first_bad_version(20, is_bad), 7)

assert_equal(search_rotated([4,5,6,7,0,1,2], 0), 4)
assert_equal(search_rotated([4,5,6,7,0,1,2], 3), -1)

assert_equal(kth_largest([3,2,1,5,6,4], 2), 5)

assert_equal(merge_intervals([[1,3],[2,6],[8,10],[15,18]]), [[1,6],[8,10],[15,18]])
print("[ok] Practice problem tests passed.")


[ok] Practice problem tests passed.


## 5) Stability & In-Place Discussion
Answer briefly in Markdown:
1. Which of the four sorts you wrote are stable? Which are in-place?
2. Give an example where stability affects correctness.
3. Why does quicksort have bad worst-case behavior and how to mitigate it in practice?


_Your notes here..._


## 6) Optional Extensions (Bonus)
- **Counting Sort / Radix Sort** for integers with bounded ranges.
- **Binary Search Variants**: lower_bound / upper_bound positions.
- **Order Statistics**: deterministic linear-time selection (median of medians).
- **Stability Experiment**: craft inputs to visually demonstrate stability vs. instability.


In [13]:
def lower_bound(arr: List[int], target: int) -> int:
    """First index i such that arr[i] >= target. If not found, return len(arr)."""
    lo, hi = 0, len(arr)
    while lo < hi:
        mid = (lo + hi) // 2
        if arr[mid] < target:
            lo = mid + 1
        else:
            hi = mid
    return lo

def upper_bound(arr: List[int], target: int) -> int:
    """First index i such that arr[i] > target. If not found, return len(arr)."""
    lo, hi = 0, len(arr)
    while lo < hi:
        mid = (lo + hi) // 2
        if arr[mid] <= target:
            lo = mid + 1
        else:
            hi = mid
    return lo

# quick demo
a = [1,2,2,2,3,5]
print('lower_bound for 2:', lower_bound(a, 2))  # 1
print('upper_bound for 2:', upper_bound(a, 2))  # 4


lower_bound for 2: 1
upper_bound for 2: 4


---
### Submission Checklist
- [ ] All `Run tests` cells pass
- [ ] You answered the short discussion questions in Section 5
- [ ] Commit & Push your changes
- [ ] Open a PR titled `Algorithms Lab: Searching & Sorting`

Good luck! 🚀
